In [7]:

from langchain_community.vectorstores import Weaviate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOllama

import weaviate
from langchain.globals import set_llm_cache
from langchain.cache import RedisCache
import redis

REDIS_URL = "redis://localhost:6379/0"

redis_client = redis.Redis.from_url(REDIS_URL)
set_llm_cache(RedisCache(redis_client))


client = weaviate.Client(
url="http://localhost:8080",
)

vectorstore = Weaviate(client, 
                    "GRP", 
                    "content")

retriever = vectorstore.as_retriever()



# RAG prompt
template = """You're an Friendly AI assistant, your name is Claro, you can make normal conversations in a friendly manner, and also provide Answer the question based on the following context make sure it sounds like human and official assistant:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)




# RAG
model = ChatOllama(model="openhermes:7b-mistral-v2-q8_0")
# model = ChatOllama(model="falcon:40b-instruct-q4_1")
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    # | StrOutputParser()
)



# Add typing for input
class Question(BaseModel):
    __root__: str


chain = chain.with_types(input_type=Question)



/home/amaithi/anaconda3/envs/lang/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.1. The latest version is 4.5.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [8]:
result = chain.invoke('what is mediwave')

In [9]:
result

AIMessage(content="Mediwave Digital is a healthcare technology and clinical research company that focuses on combining user-centered design with agile development. They specialize in creating digital products and services that deliver delightful customer experiences, while also addressing the world's needs and making a meaningful impact. Mediwave works closely with clients to understand their needs and provide tailored solutions, ensuring satisfaction throughout the customer journey.")

In [ ]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [ ]:
from grp_food_crew_ai.grp_food_crewai import food_crew


result = food_crew(input='how to make vanilla sponge cake give me the receipe only')

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ig

 [DEBUG]: Working Agent: Crew Manager
 [DEBUG]: [Crew Manager] Task output: None
{'final_output': None, 'tasks_outputs': [None]}


> Entering new CrewAgentExecutor chain...


 I should delegate the task of finding and providing the recipe for a vanilla sponge cake to the Ingredient Expert since they have extensive knowledge about ingredients and their usage in various recipes.

Action: Delegate work to co-worker
Action Input: {"coworker": "Ingredient Expert", "task": "Find and provide the recipe for a vanilla sponge cake", "context": "This is a request from a user who wants to make a vanilla sponge cake. They only want the receipe."} 


Error executing tool. Co-worker mentioned not found, it must to be one of the following options:
- Master Chef
- Nutrition Expert
- Ingridient Expert
- Image analyzing Expert
- grocery Expert
- wine analyzing Expert
- food joke and trivia Expert 
- Head of the crew


 Thought: I apologize for the error in my previous action. I should have delegated the task to the Master Chef instead, as they are responsible for preparing and cooking various dishes, including vanilla sponge cake.

Action: Delegate work to co-worker
Action In

/home/amaithi/projects/weaviate/langgraph-final/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


 

ERROR making request

 Thought: It seems I encountered an error while trying to use Spoonacular API's searchRecipesByIngredients tool. Let me try using a different tool or provide a recipe from my knowledge base.

Action: None
Action Input: None 

Action 'None' don't exist, these are the only available Actions: spoonacular_API.searchRecipes: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: Search through hundreds of thousands of recipes using advanced filtering and ranking. NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.
spoonacular_API.searchRecipesByIngredients: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: 
Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or minimize

/home/amaithi/projects/weaviate/langgraph-final/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/amaithi/projects/weaviate/langgraph-final/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


 

Please provide an API_RESPONSE for me to process and create a human-understandable synthesis.

 Thought: I need to wait for the API response before I can create a human-understandable synthesis of the search results.

Action: None
Action Input: None 

Action 'None' don't exist, these are the only available Actions: spoonacular_API.searchRecipes: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: Search through hundreds of thousands of recipes using advanced filtering and ranking. NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.
spoonacular_API.searchRecipesByIngredients: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: 
Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or mini

In [2]:
print(result)

{'final_output': None, 'tasks_outputs': [None]}


In [4]:
from grp_food_crew_ai.grp_food_crewai import food_crew


result = food_crew(input='how to make vanilla sponge cake, give me the receipe')

Overriding of current TracerProvider is not allowed


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 				**Task**: find and provide the appropriate soultion to the user needs.
           **Description**: you are an helpfull assuistane.    
            **Parameters**: 
            - user need: how to make vanilla sponge cake, give me the receipe


> Entering new CrewAgentExecutor chain...


In [2]:
print(result)

To make this chocolate cake, first preheat the oven to 350°F (180°C) and grease an 8-inch round cake pan with cooking spray or butter. In a medium bowl, whisk together flour, cocoa powder, baking powder, baking soda, and salt. In a large bowl, beat sugar and oil until combined, then add the egg and mix well. Stir in milk and vanilla extract. Gradually add the dry ingredients to the wet mixture, stirring until just combined. Slowly pour in boiling water and stir until the batter is smooth. Pour the batter into the prepared cake pan and bake for 30-35 minutes or until a toothpick inserted into the center comes out clean. Let the cake cool before serving. To ensure the best results, use fresh ingredients and follow the recipe's instructions carefully.


Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values 

RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
a = 5 
a

5

# TRavel